# Monte Carlo Tests on Sigma Sys
# Here sigma_sys is an overall amplitude offset as described in the paper

1) test recovery of Sigma-sys
2) see what value Sigma_sys corresponds to (it isn't just a straight average)

## Part 1 : test sigma-sys recovery scheme

In [1]:
import numpy as np
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy import stats
from scipy import optimize

%matplotlib inline  

In [2]:
# The HOD model file
# see interpolate.pro
modelfile = "hod_model_saito2016_mdpl2.txt"       # model for weighting
temp = np.genfromtxt(modelfile)
model = temp[:,1]
rbins = temp[:,0]
rbins

array([ 0.0692234,  0.122452 ,  0.216609 ,  0.383168 ,  0.6778   ,
        1.19898  ,  2.12093  ,  3.75178  ,  6.6366701, 11.7398005])

In [3]:
smallr ,= np.where(rbins<0.1) # 1 Mpc split

larger ,= np.where(rbins>1.)
allr ,= np.where(rbins<50)

binranges = []
binranges.append(smallr)
binranges.append(larger)
binranges.append(allr)

nrbin=len(rbins)
smallr

array([0])

In [4]:
# Function that computes amplitudes

def amplitudes(measurements, covariances, model, binranges, experiments, file1, file2, file3, file4):
    # measurements is a dictionary of Nd vectors, 
    # covariances is a dictionary of NxN matrixes, 
    # model is a Nd vector, 
    # binranges is a list of integer vectors, 
    # experiments is a list of names 
    
    for binrange in binranges:
        
        #print("------ Bin range -----",binrange)
        #print("bin range ",binrange)
        
        # pick relevant entries from measurements, covariances, model
        meas = {}
        cov = {}
        invcov = {}
        weight = {} # optimal weight for that survey
        denom  = {} # the denominator for that survey that brings A=1 if meas=mod
        mod = model[binrange]
        #print("model",mod)
        
        # sum of covariances of that set of bins
        sumcov = np.zeros((len(binrange),len(binrange)))
        
        #print("\n----- Calculating weights -------\n")
            
        for experiment in experiments:
            
            #print(experiment)
            meas[experiment] = measurements[experiment][binrange]
            cov[experiment]  = covariances[experiment][binrange][:,binrange]
            invcov[experiment] = np.linalg.inv(cov[experiment])
            weight[experiment] = np.dot(invcov[experiment],mod)
            #print("experiment",experiment)
            #type(weight[experiment])
            #print("weight",weight[experiment])
            
            denom[experiment] = np.dot(weight[experiment],mod)
            
            # sum all covs for joint weighting
            sumcov += cov[experiment]
            
        # optimal weight for comparison is inverse of sum of covariances times model
        cweight = np.dot(np.linalg.inv(sumcov),mod)
        cdenom   = np.dot(cweight,mod) # normalizing the amplitude to 1 if measurement = model
        
        #print("cweight",cweight)
        #print("cdenom",cdenom)      
        
        #print("\n----- Calculating amplitudes -------\n")
        
        for experiment in experiments:
            a    = np.dot(meas[experiment],cweight)/cdenom
            siga = np.sqrt(np.dot(np.dot(cweight,cov[experiment]),cweight)/cdenom**2)
            #print(experiment,a,"+-",siga)
            
            # Write out amplitudes
            file1.write('%0.4f' % a)
            file1.write(' ')

            # Write out errors
            file2.write('%0.4f' % siga)
            file2.write(' ')
        
            #print("a and siga", a, siga)
   
            #print("with its own optimal weighting the experiment",experiment,"would have given us",a,"+-",siga)
            a    = np.dot(meas[experiment],weight[experiment])/denom[experiment]
            siga = np.sqrt(np.dot(np.dot(weight[experiment],cov[experiment]),weight[experiment])/denom[experiment]**2)
   
        # also print out the weights
        # to check if they have imporatnt variations with R
        #print('cweight',cweight)
        
        for item in cweight:
            file3.write("%0.4f" % item)
            file3.write(' ')
            
        file4.write('%0.4f' % cdenom)
                
        file1.write('\n')
        file2.write('\n')
        file3.write('\n')
        file4.write('\n')
            
    file1.close()  
    file2.close() 
    file3.close()
    file4.close()

In [5]:
experiments = ["e1","e2","e3","e4","e5","e6"] 

In [6]:
# Provide data and errors, and sigma_sys, return reduced chi2-1 with weighed mean
# Data and errors passed as globals
def computechi2minus1(sigs):
    
    global data2
    global errs2
    
    n=len(data2)
    
    # inverse variance mean
    w = np.zeros(n)
    for i in range(0, n):
        w[i]=1.0/(errs2[i]**2)
    
    wmean = np.average(data2, weights=w)
    
    #Compute reduced chi2
    chi2=0.0

    for i in range(0, n):
        chi2+=((data2[i]-wmean)**2)/(errs2[i]**2+sigs**2)
    
    rchi2 = chi2/(n-1)

    # Reduced chi2 -1
    return rchi2-1

# Connection between error on amplitude and systematic error 

# Run Monte Carlo Test

In [7]:
# Change input parameters to run new test
# One common statistial error assumed here

# Staterr = 0.05   # Statistial in percentage
# Systematic error in percentage, but this is one overall 
# offset (same per radial bin)
#    syserr = 0.1

def runnewtest(staterr,syserr):

    measurements = {}
    covariances = {}

    k=0    
    for experiment in experiments:
    
        #Data Vector
        modelfile = "hod_model_saito2016_mdpl2.txt"       # model for weighting
        temp = np.genfromtxt(modelfile)
        dstemp = temp[:,1]
    
        # Generate cov that has **statistical** only
        covtemp = np.genfromtxt("SDSS_LOWZ_0.15_0.31_wtot_cov.dat")
        for i in range(0,nrbin): 
                for j in range(0,nrbin): 
                    if (i == j):
                        covtemp[i,i] = dstemp[i]*staterr # error in percentage
                    else:
                        covtemp[i,j]=0.0
        #print(covtemp)
        #print("----")
    
        # Now add statistical and systematic error to data vecor
        # Statistical is drawn each time anew for each radial bin
        # systematic is an offset that is the same in all bins
        
        this_sys_draw = np.random.normal(0,syserr,1)[0]
        
        for i in range(0,nrbin): 

            #and statistical error 
            sig = dstemp[i]*staterr
            
            # systematic offset 
            dstemp[i] = dstemp[i] + dstemp[i]*this_sys_draw 
            
            # Statistical, new draw for each bin
            dstemp[i] = dstemp[i]+np.random.normal(0,sig,1)[0]
       
        
        #print(model)
        #print(dstemp)
        #print("----")
    
        covariances[experiment] = covtemp
        measurements[experiment]= dstemp
    
        k=k+1
    
    # Now run the Amplitudes
    file1 = open("testamplitudes.txt",'w')
    file2 = open("testamplitudes_errs.txt",'w')
    file3 = open("testamplitudes_cweight.txt",'w')
    file4 = open("testamplitudes_cdenom.txt",'w')
    
    amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)    

In [8]:
# runnewtest(staterr,syserr):
runnewtest(0.05,0.1)

In [ ]:
# Change numbers in runnewtest to check different combinations

ntest=5000

sigest = np.zeros(ntest)

for k in range(0,ntest):   # Run the test many many times
    
    runnewtest(0.05,0.1)
    allamp = np.loadtxt("testamplitudes.txt")
    allamperrs = np.loadtxt("testamplitudes_errs.txt")
    
    # All Radial range used here
    # Change here to use a different radial range
    #print("----")
    data=allamp[0]
    errs=allamperrs[0]
    
    n=len(data)
    data2 = np.zeros(n)
    errs2 = np.zeros(n)
    
    w = np.zeros(n)
    for i in range(0, n):
        w[i]=1.0/(errs[i]**2)
    
    wmean = np.average(data, weights=w) 
    meanamp = np.average(data, weights=w) 
        
    #Compute reduced chi2
    chi2=0.0

    for i in range(0, n):
        chi2+=((data[i]-wmean)**2)/(errs[i]**2)
    
    rchi2 = chi2/(n-1)
    #print("Reduced chi2: ", rchi2)
    
    meanstaterr = np.mean(errs)
    
    # If reduces chi2 >1 then proceed 
    if (rchi2>1):
        
        # Find the value that makes rchi2=1
        # Set global variables
        for i in range(0, n):
            data2[i]=data[i]
            errs2[i]=errs[i]

        sol=optimize.root_scalar(computechi2minus1, bracket=[0,2.0])
        sigmasys=sol.root
        sigest[k]=sigmasys
              
    # --------------------------     
    # Confidence interval for sigma_sys
    # this is a frequentist thing -- at what values of sigma_sys 
    # (including 0) do the differences we find between the surveys produce a chi^2 that is within the central
    # one and two sigma of the distribution
    #--------------------------

    ndof = len(data)-1 # should be chi^2-distributed with this ndof, since fitting for mean

    Dsigma_sys = np.arange(0,0.6,0.01) # range of sigma_sys to probe

    Dchisq = np.zeros(len(Dsigma_sys)) 
    # the chi^2 we would have if we included sigma_sys mutually independent systematic error
    for i in range(len(Dsigma_sys)):
        Dchisq[i] = np.sum((data-wmean)**2/(errs**2+(Dsigma_sys[i])**2))
    
    # ---- 1 Sigma Limits -----
    # inverse of 1-CDF 
    xx=(1.0-0.68)/2.0 # 68 confidence
    # inverse survival function
    limits = scipy.stats.chi2.isf(q=[1.0-xx,xx],df=ndof)
    #print("68% confidence interval for chi^2 (in chi2 space) is", limits)

    # Confidene interval for the observed chi2
    ok ,= np.where(np.logical_and(Dchisq>limits[0],Dchisq<limits[1]))
 

    #print("68% confidence interval on sigma sys is",Dsigma_sys[ok[0]],Dsigma_sys[ok[-1]])
        
#plt.hist(sigest,bins=20,range=[0,0.25])

In [ ]:
y=1000
plt.hist(sigest,bins=20,range=[0,0.30])
#plt.plot([sigsys,sigsys],[0,y])
plt.plot([np.mean(sigest),np.mean(sigest)],[0,y])
plt.plot([np.median(sigest),np.median(sigest)],[0,y])
#sigest